In [1]:
import numpy as np
import os
import fabio
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import re

from pyFAI.multi_geometry import MultiGeometry
from pyFAI.azimuthalIntegrator import AzimuthalIntegrator as AzInt

In [2]:
def load_image(imageFullname):
    """
    author: Fang Ren (SSRL), Robert Tang-Kong
    version: 1.1

    returns: numpy array with image data
    CURRENTLY HARD CODED FOR PILATUS
    """
    # get extension to consider
    ext = os.path.splitext(imageFullname)[1]
    #print(imageFullname)
    imArray = np.array([])
    if ext.lower() in ['.tif', '.tiff']:
        # open tiff image
        im = fabio.open(imageFullname)
        # input image object into a numpy array
        imArray = im.data
    elif ext.lower() in ['.raw']:
        # extract raw file
        im = open(imageFullname, 'rb')
        arr = np.fromstring(im.read(), dtype='int32')
        im.close()

        #raw requires prompting for dimensions, hard code for now
        arr.shape = (195, 487)
        imArray = np.array(arr)

    return imArray

In [3]:
# Organize paths and collect images to integrate (data)
base_path = Path('C:\\Users\\roberttk\\Desktop\\SLAC_RA\\DATA\\July2021_Xiaolin\\XiaolinZhang\\')
exportPath = base_path / 'LaoFTO1_export'
specPath = base_path / 'LaoFTO1' # folder holding spec files

specFiles = specPath.glob('*.csv') # use re to be more precise
specs = [pd.read_csv(x, index_col=0).rename(columns=lambda x: x.strip()) for x in specFiles]
user = 'b_mehta_'
imgPath = specPath / 'images'

# define detector.  
from pyFAI.detectors import Pilatus100k
det = Pilatus100k()

# define detector geometry
# USER MUST CHANGE THIS VALUE
centerPx1, centerPx2 = (91, 250)

In [4]:
testPath = imgPath / 'b_mehta_LAOFTO1_Gi0p1_scan1_0064.raw'
print(testPath)
img = load_image(testPath)

# if detector is portrait mode, need to rotate images
# plt.imshow(np.rot90(img, k=1))#[:, 100:400])
# plt.scatter(91, 277, c='r')

# if detector is landscape, can use as is
plt.imshow(img)
plt.scatter(250, 91, c='r')

C:\Users\roberttk\Desktop\SLAC_RA\DATA\July2021_Xiaolin\XiaolinZhang\LaoFTO1\images\b_mehta_LAOFTO1_Gi0p1_scan1_0064.raw


<ipython-input-2-3097903a4635>:21: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  a1rr = np.fromstring(im.read(), dtype='int32')


NameError: name 'arr' is not defined

In [ ]:
plt.figure()
# if landscape
mask = np.ones(np.shape(img))
mask[:, 100:400] = 0

# # if portrait
# mask = np.ones(np.shape(np.rot90(img, k=3)))
# mask[100:400, :] = 0 # zero's are valid

plt.imshow(mask)
plt.show()

In [ ]:
imgGlob = imgPath.glob('*LAOFTO1_Gi0p1_scan1*.raw')

spec = pd.read_csv(list(specPath.glob('*LAOFTO1_Gi0p1_scan1.csv'))[0])

spec.rename(columns=lambda x: x.strip(), inplace=True)

print(specPath)
# print(len(list(imgGlob)))
imgs = []
ais = []

#imgGlob = list(imgGlob)[:4]

for rawPath in imgGlob:
    # Append img to list of images
    img = load_image(rawPath)
    
    imgs.append(np.rot90(img, k=1))

    # Append calibrated ai to list of ai's
    scanNo = int(re.search('(\d{4})(\.raw)', str(rawPath)).group(1))
    ai = AzInt(dist=0.714728, #m 
               poni1=centerPx2*det.pixel1, poni2=centerPx1*det.pixel1, detector=det, 
               wavelength=7.29319E-11, 
               rot2=np.pi/180*float(spec['tth'][scanNo]), # tth rotation
               rot3=-np.pi/2) # For bringing azimuth to 0
    ais.append(ai)

mg = MultiGeometry(ais, unit="2th_deg", radial_range=(5, 50), azimuth_range=(-30, 30))
fig, ax = plt.subplots(2, 1, figsize=(8,12))

int1d = mg.integrate1d(imgs, 10000, lst_mask=mask)
ax[0].plot(int1d[0], int1d[1])
ax[0].set_ylabel('Intensity')
ax[0].set_xlim(0,65)

res2d = mg.integrate2d(imgs, 1000, 360, lst_mask=mask)
# gives 3 arrays.  First is intensities with shape (arg2, arg1)  Second is 2th range, third is azimuthal range

ax[1].pcolormesh(res2d[1], res2d[2], np.log(res2d[0]), cmap = 'viridis')

ax[1].set_xlim(0,65)
ax[1].set_xlabel('Radial Angle 2theta')
ax[1].set_ylabel('Azumuthal Angle chi')